# Advanced AD application
This notebook documents how the automatic (or algorithmic) differentiation framework may be applied to non-linear equations. For an introduction to the framework, see the [automatic differentiation](./automatic_differentiation.ipynb) notebook.

The functions in question are the normal and tangential complementary equations for contact mechanics, which are only semi-smooth (i.e. they are not differentiable everywhere):

\begin{equation}
\begin{aligned}
C_n &= \lambda_n + \text{max}(0, -\lambda_n-c_n([[u]]_n-g))\\
C_{\tau} &= \text{max}(0, b) (\lambda_{\tau}+c_{\tau}[[\dot{u}]]_{\tau})
- \text{max}(b, ||\lambda_{\tau}+c_{\tau}[[\dot{u}]]_{\tau}||)\lambda_{\tau},
\end{aligned}
\end{equation}
with $b=-F(\lambda_n+c_n([[u]]_n-g))$ and F, c, and $g$ denoting friction coefficient, numerical constants and the gap function, respectively. See [Hüeber 2008](https://elib.uni-stuttgart.de/handle/11682/4854) for a detailed derivation and discussion and [Stefansson et al. 2021](https://www.sciencedirect.com/science/article/pii/S0045782521004539) for notation.

## Implementation
The implementation is found within the `MomentumBalance` class. After defining subdomain and interface ad variables, `set_equations` calls the methods `tangential_fracture_deformation_equation` and `normal_fracture_deformation_equation` which compose the equations from subcomponents defined in other methods:

In [1]:
import porepy as pp
import numpy as np
import inspect

model = pp.momentum_balance.MomentumBalance({})
print(inspect.getsource(model.set_equations)) 

    def set_equations(self):
        """Set equations for the subdomains and interfaces.

        The following equations are set:
            - Momentum balance in the matrix.
            - Force balance between fracture interfaces.
            - Deformation constraints for fractures, split into normal and tangential
              part.

        See individual equation methods for details.

        """
        matrix_subdomains = self.mdg.subdomains(dim=self.nd)
        fracture_subdomains = self.mdg.subdomains(dim=self.nd - 1)
        interfaces = self.mdg.interfaces(dim=self.nd - 1)
        matrix_eq = self.momentum_balance_equation(matrix_subdomains)
        # We split the fracture deformation equations into two parts, for the normal and
        # tangential components for convenience.
        fracture_eq_normal = self.normal_fracture_deformation_equation(
            fracture_subdomains
        )
        fracture_eq_tangential = self.tangential_fracture_deformation_equation(
     

The simpler of the equations is defined as follows:

In [2]:
print(inspect.getsource(model.normal_fracture_deformation_equation))

    def normal_fracture_deformation_equation(self, subdomains: list[pp.Grid]):
        """Equation for the normal component of the fracture deformation.

        This constraint equation enforces non-penetration of opposing fracture
        interfaces.

        Parameters:
            subdomains: List of subdomains where the normal deformation equation is
            defined.

        Returns:
            Operator for the normal deformation equation.

        """
        # The lines below is an implementation of equations (24) and (26) in the paper
        #
        # Berge et al. (2020): Finite volume discretization for poroelastic media with
        #   fractures modeled by contact mechanics (IJNME, DOI: 10.1002/nme.6238). The
        #
        # Note that:
        #  - We do not directly implement the matrix elements of the contact traction
        #    as are derived by Berge in their equations (28)-(32). Instead, we directly
        #    implement the complimentarity function, and

## Non-smooth functions using pp.ad.Function
Handling non-smoothness in the AD setting requires the definition of extended derivatives by assigning appropriate values to the Jacobi matrices for the non-smooth function components ($\text{max}$ and $\text{abs}$) at the points in question. While this may seem somewhat technical, it is a modest price to pay for handling these equations otherwise straightforwardly using AD. We define standard Python functions and wrap them in `pp.ad.Function` returning `pp.ad.Ad_array`s having a val and a jac attribute. For instance, the maximum value function is defined and used as follows:

In [3]:
print(inspect.getsource(pp.ad.functions.maximum)) 

def maximum(
    var_0: pp.ad.Ad_array, var_1: pp.ad.Ad_array | np.ndarray
) -> pp.ad.Ad_array:
    """Ad maximum function represented as an Ad_array.

    The arguments can be either Ad_arrays or ndarrays, this duality is needed to allow
    for parsing of operators that can be taken at the current iteration (in which case
    it will parse as an Ad_array) or at the previous iteration or time step (in which
    case it will parse as a numpy array).


    Parameters:
        var_0: First argument to the maximum function.
        var_1: Second argument.

        If one of the input arguments is scalar, broadcasting will be used.


    Returns:
        The maximum of the two arguments, taken element-wise in the arrays. The return
        type is Ad_array if at least one of the arguments is an Ad_array, otherwise it
        is an ndarray. If an Ad_array is returned, the Jacobian is computed according to
        the maximum values of the Ad_arrays (so if element ``i`` of the maximum is
   

## Technical notes on Function wrapping
### Argument types
The wrapping of a function in the pp.ad.Function class may be slightly confusing in that the function (e.g. `pp.ad.functions.max`) takes an `Ad_array` as its argument, whereas the Function instance (e.g. `MaxAd` above) expects an `Operator`, which represents an ad variable or compound expression. The explanation lies in how the Function is *parsed* ("evaluated"), which involves the `MaxAd` asking its `_function` to operate on the values and jacobians of `var0` and `var1`, which are represented through an `Ad_array`. Puh!

### Chain rule
An ad `Funtion` is parsed as follows by `pp.ad.Operator._parse_operator`:
```
elif tree.op == Operation.evaluate:
    # This is a function, which should have at least one argument
    assert len(results) > 1
    return results[0].func(*results[1:])
```
That is, it calls the wrapped function on the ad array produced by parsing of the function argument(s). This means that the chain rule should be applied internally in the function. For a generic funtion `f` of a single variable `var` with derivative `f_prime` with respect to `var`, we have
```
def function_to_be_wrapped(var: pp.ad.Ad_array) -> pp.ad.Ad_array:
    var = f(var)
    df_dvar = f_prime(var)
    # Chain rule:
    jac = var.diagvec_mul_jac(df_dvar)
    return  pp.ad.Ad_array(var, jac)
```

### Partial functions
Some functions depend on arguments which do not have anything to do with ad. Instead of having to wrap such arguments in AD objects to be evaluated as part of parsing of the Function, one can exploit partial evaluation. For instance, the `pp.ad.functions.l2_norm` function for cell-wise vectors has been implemented for an arbitrary number of vector components. It is applied in the definition of the gap, which depends on the norm of tangential displacement jumps. The number of tangential components equals the dimension of the fracture, i.e. $nd - 1$:

In [4]:
print(inspect.getsource(model.gap))

    def gap(self, subdomains: list[pp.Grid]) -> pp.ad.Operator:
        """Fracture gap [m].

        Parameters:
            subdomains: List of subdomains where the gap is defined.

        Returns:
            Cell-wise fracture gap operator.

        """
        angle: pp.ad.Operator = self.dilation_angle(subdomains)
        f_norm = pp.ad.Function(
            partial(pp.ad.functions.l2_norm, self.nd - 1), "norm_function"
        )
        f_tan = pp.ad.Function(pp.ad.functions.tan, "tan_function")
        shear_dilation: pp.ad.Operator = f_tan(angle) * f_norm(
            self.tangential_component(subdomains) * self.displacement_jump(subdomains)
        )

        gap = self.reference_gap(subdomains) + shear_dilation
        gap.set_name("gap_with_shear_dilation")
        return gap



# What have we explored
We have seen how the Ad framework can be used to specify non-linear functions, including ones that are sub-differentiable but not fully differentiable (e.g., the maximum function), and reviewed this in the context of contact mechancis. The model classes implement the functions and constitutive relations most relevant for standard applications of PorePy. New Ad functions can easily be implemented by wrapping standard python functions using `pp.ad.Function`.